In [1]:
import sys
import numpy as np
import pickle as pkl
from IPython.display import display
from maraboupy import Marabou, MarabouCore
from utils import TOTUtils

In [2]:
sample_input = [
    0.2622614500364243, -0.22479658685112885, -0.14872559153186365,-0.1183740939370412,       -0.11582643087693376, 0.4638055359970424, 0.5343624908151317, -1.8587238209603285,        -1.925346908225373, 0.7745258391529399, 1.691414092773877, -1.1513317010900714,           0.15687995017838718, -0.4932713694542802, 0.3758603164252629, -0.6443093237643779,        0.5198151301903109, -0.1392841543884293, -0.17885723928180608, -0.7026510235427964,       -0.04793722518544871, -0.4594806526602232, 0.18987588687321624, 0.4657886911920623,       -0.1626985247701109]
sample_output = [1, 0, 0]
samples = [(sample_input, sample_output)]

nnet_path = '../artifacts/models/model-v3.1.3/model.nnet'


from sensitivity import find_feature_sensitivity_distances
results = {}
for x in range(25):
    r = find_feature_sensitivity_distances(nnet_path, x, samples)
    print(f'x{x}', r)
    results[f'x{x}']=r
print('RESULTS: ', results)

D 0.01 P 0 R UNSAT
A
D 0.03 P 0.01 R UNSAT
A
D 0.06999999999999999 P 0.03 R UNSAT
A
D 0.14999999999999997 P 0.06999999999999999 R UNSAT
A
D 0.30999999999999994 P 0.14999999999999997 R UNSAT
A
D 0.6299999999999999 P 0.30999999999999994 R UNSAT
A
D 1.2699999999999998 P 0.6299999999999999 R UNSAT
A
D 2.55 P 1.2699999999999998 R UNSAT
A
D 5.109999999999999 P 2.55 R SAT
C
D 3.8299999999999996 P 5.109999999999999 R UNSAT
B
D 4.47 P 3.8299999999999996 R UNSAT
A
D 5.75 P 4.47 R SAT
C
D 5.109999999999999 P 5.75 R SAT
D
D 4.789999999999999 P 5.109999999999999 R UNSAT
B
D 4.949999999999999 P 4.789999999999999 R SAT
C
D 4.869999999999999 P 4.949999999999999 R UNSAT
B
D 4.909999999999999 P 4.869999999999999 R UNSAT
A
D 4.989999999999999 P 4.909999999999999 R SAT
C
D 4.949999999999999 P 4.989999999999999 R SAT
D
D 4.93 P 4.949999999999999 R SAT
D
D 4.92 P 4.93 R UNSAT
B
D 4.925 P 4.92 R SAT
C
D 4.922499999999999 P 4.925 R UNSAT
B
D 4.92375 P 4.922499999999999 R UNSAT
A
D 4.926250000000001 P 4.92375 

(-4.9248287433600595, 1.1605207445844776)

In [3]:
# Max UNSAT distances per feature for sample_input
# distances = [0.0] * len(sample_input)
# distances[0] = 1.16
# distances[1] = 0.37
# distances[2] = 0.18
# distances[3] = 1.48
# distances[4] = 0.31
# distances[5] = 0.92
# distances[6] = 0.27
# distances[7] = 1.89
# distances[8] = 1.61
# distances[9] = 6.85
# distances[10] = 25.53
# distances[11] = 0.40
# distances[12] = 1.09
# distances[13] = 0.22
# distances[14] = 1.21
# distances[15] = 1.32
# distances[16] = 0.66
# distances[17] = 0.39
# distances[18] = 1.04
# distances[19] = 0.17
# distances[20] = 6.20
# distances[21] = 1.15
# distances[22] = 0.42
# distances[23] = 2.82
# distances[24] = 0.81

In [4]:
# from sensitivity import evaluate_sample
# x=0
# d=1.17
# s=sample_input[0:x] + [sample_input[x]+d] + sample_input[x+1:]
# evaluate_sample(nnet_path, s, sample_output)

In [5]:
# from robustness import find_feature_sensitivity_bruteforce
# find_feature_sensitivity_bruteforce(nnet_path, 0, samples)

In [6]:
# from sensitivity import find_feature_sensitivity_binary_search

# find_feature_sensitivity_binary_search(nnet_path, 0, samples)

In [7]:
# from sensitivity import test_network_sensitivity
# test_network_sensitivity(nnet_path, len(samples[0][0]), samples)